In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/09 16:19
# @Author  : Wang Yujia
# @File    : SA_for_PT_model.ipynb
# @Description : Parameter estimation for PT_model using Simulated Annealing and Delta==1

# 0. What for

# 1. Preparations
1. infer参数一是需要data，二是需要把p表示出来才能写出来loss func
2. data来自`data_selected_path`

## 1.1 全局设置
1. 除了表示uniq auction的features，还引入了
    - 'cnt_uniq':表示paper里的Loss function公式里的A

In [9]:
# data path
# data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
data_selected_path = "../../data/SA_PT/datawithnp_PT_selected.csv"

# data_key path
data_key_path = "../../data/SA_PT/data_key_PT.csv"
# optimized parameters' saving path:
params_opitim_delta_path = "../../data/SA_PT/params_opitim_delta_T.csv"
params_opitim_delta_wset_path = "../../data/SA_PT/params_opitim_delta_T_wset.csv"

# features that GT need
features_GT = ['product_id','bidincrement','bidfee','retail']
features_GT_infer = ['cnt_uniq']

# for SA
# initial params
table_5_M = [0.025,3.72]
# table_5_M = [0,2]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]

import numpy as np
import pandas as pd
import seaborn as sns
from SA_modified import SABoltzmann
from SA_for_PT_funcs_delta_eq1 import *
import matplotlib.pyplot as plt
import datetime
from sko.tools import set_run_mode
from visdom import Visdom
from tqdm import tqdm

# viz = Visdom(env = "Loss of PT_2")

## 1.2 data 读取
1. 读取data以做SA
2. 提取出来`data_key`，以及其他计算需要的features

In [7]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
data_key = data[features_GT].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
data_key.to_csv(data_key_path,header=True, encoding="utf-8",index=False)

# 需要计算`N_uniq_auction`组setting下的结果
N_uniq_auction = data_key.shape[0]

print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))
data_key.head()

For PT model, there are *1400* settings waiting to be inferred.


,product_id,bidincrement,bidfee,retail
0,10009602,0.15,0.75,499.99
1,10009881,0.15,0.75,169.99
2,10006115,0.15,0.75,499.99
3,10008975,0.15,0.75,499.99
4,10007148,0.15,0.75,299.99


## 1.3 functions about 'key'

In [5]:
# get key from i in 'data_key'
def get_key_from_index(i,str="NotStr"):
    if(str=="str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(key_i[0]),str(key_i[1]),str(key_i[2]))
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()


# 2. Equi. condition in PT model
1. 根据Eq(6)
2. 注意分辨怎么代入上面的公式
3. `delta = 1`时，公式可以大大化简，见ipad上的公式

In [6]:
def f_Equi(t,v,d,b,alpha,labda):

    tmp = v-d*t-C(t-1,b) - b

    if (tmp>=0):
        root = (labda*f(C(t-1,b),alpha) + f(tmp,alpha)) / (labda*f(C(t-1,b)+b,alpha) + f(tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} ---- u = 0.0:{root} ---- alpha : {alpha}")
    else:
        # print("tmp starts to < 0", t)
        root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} -- u = 0.0:{root} -- alpha : {alpha} -- lamda : {labda}")

    #viz.line([[0.0,0.0]],[0],win = 'root compare',opts= dict(title='root in 2 methods'+str(t),legend=['simplify', 'sympy']))
    #viz.line([[np.float(root1),np.float(root)]],[t],win = 'root compare', update='append')

    return root

# 3. SA
## 3.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2. 注意只能使用理论上限max_T之内的samples

In [23]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]

    max_T,v,d,b,T_i,cnt_n_2_i = other_params    # T_i: auction actual duration sequence
    # max_T = int(max(T_i))                     # max_T: max duration value
    T = int((v-b)/d)                            # duration upper bound
    flag = 1    # ?
    #
    # Solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1
    for t in range(1,max_T+1):
        U_i[t] = f_Equi(t, v, d, b, alpha, labda)
        if(flag & (U_i[t]<=0)):
            print(f"t:{t} -- u starts to <= 0.0: {U_i[t]} -- alpha : {alpha} -- lamda : {labda}")
            flag = 0

    # Calculate NLL under this auction setting & PT params
    nll = 0.0
    if(U_i[0]==1):
        U_i.pop(0)
    for idx in range(0,data_i.shape[0]):
        # If duration succeeds the theory bound T
        if T_i[idx] > T:
            break
        # Sum up the log prob among all durations of this auction
        nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]
    # print('> The loss costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time_loss).total_seconds()))

    return float(-nll)

## 3.2 do SA
1. 要对每一个setting做一次infer == 对每一个setting执行一次SA。
    - 可以并行吗？YES
2. 具体的：对每个setting `i`
    - 每一个setting `i` 对应一组samples: `data_i`
    - `data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
3.每次进行`L`次对参数的试探寻找，每次寻找对应一个温度一组新的参数。
    - 优化的完成/退出条件：温度小于`T_min`或者最低温度保持`max_stay_counter`次的不变
    - 鉴于温度小于`T_min`很难达到，因此基本上对一组参数进行优化要进行L*max_stay_counter+1次运算（loss运算）

In [24]:
params_opitim = pd.DataFrame(columns=['key_idx','alpha','delta','labda','initial_loss','final_loss','avg_loss'])

In [25]:
# Perform SA respectively for all settings
# Fail: [30,124,151,175,235,331,416,417,524,546,572,821,940,941,1091]
# Fail 2.2:[1234]
for i in tqdm(range(N_uniq_auction)):
    #start_time = datetime.datetime.now()

    # get i_th data_key
    key_i = get_key_from_index(i)
    # extract data with same `key_i` into a table
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value
    cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
    # for a certain auction(like 'data_i'), 'cnt_uniq' should be all the same

    A_i = sum(cnt_n_2_i)        # Observation amount

    v = float(data_i['retail'].unique())
    d = float(data_i['bidincrement'].unique())
    b = float(data_i['bidfee'].unique())

    assert max_T <= (v-b)/d, "max duration of auction > T"

    # calculate NLL
    print(f"> For the *{i}_th* data_key")
    print("> retail = {0},bidincrement = {1}, bidfee = {2}, infer PT's parameters".format(v,d,b))
    print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    # L=30, max_stay_counter=15
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [max_T,v,d,b,T_i,cnt_n_2_i], T_max=round((v-d)/b), T_min=1, learn_rate=0.2, L=15, max_stay_counter=5,
                            lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()
    # print('> The whole inference process costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time).total_seconds()))

    print("> SA ENDS....... \n")
    # if(i%10 == 0):
    #     viz.line([0.0]*(sa_boltzmann.iter_cycle+1),[0]*(sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i),opts= dict(title='Loss of '+str(i)))
    #     viz.line(np.array(sa_boltzmann.generation_best_Y),np.arange(0,sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i), update='append')

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([[i,best_x[0],1,best_x[1],sa_boltzmann.generation_best_Y[0],best_y,best_y/A_i]],columns=['key_idx','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
    params_opitim = pd.concat([params_opitim,df_tmp],axis=0,ignore_index=True)  # ignore_index=True could help in rearranging index

  0%|          | 0/1400 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_37552\3277117482.py:30: RuntimeWarning: divide by zero encountered in log
  nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]


> For the *0_th* data_key
> retail = 499.99,bidincrement = 0.15, bidfee = 0.75, infer PT's parameters
> Initilizing SA....... 

> Now do SA....... 



  0%|          | 1/1400 [00:00<12:34,  1.85it/s]

STOP CODE:  Stay unchanged in the last 6 iterations
> SA ENDS....... 

> For the *1_th* data_key
> retail = 169.99,bidincrement = 0.15, bidfee = 0.75, infer PT's parameters
> Initilizing SA....... 

> Now do SA....... 



C:\Users\Administrator\AppData\Local\Temp\ipykernel_37552\3277117482.py:30: RuntimeWarning: divide by zero encountered in log
  nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]
  0%|          | 2/1400 [00:01<15:35,  1.49it/s]

...

STOP CODE:  Stay unchanged in the last 6 iterations
> SA ENDS....... 

> For the *1399_th* data_key
> retail = 1018.0,bidincrement = 0.01, bidfee = 0.6, infer PT's parameters
> Initilizing SA....... 

> Now do SA....... 

t:12533 -- u starts to <= 0.0: 0.0 -- alpha : 0.06118335477844246 -- lamda : 0.01


C:\Users\Administrator\AppData\Local\Temp\ipykernel_37552\3277117482.py:30: RuntimeWarning: divide by zero encountered in log
  nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]


t:10922 -- u starts to <= 0.0: 0.0 -- alpha : 0.09304544719102031 -- lamda : 0.01
t:12242 -- u starts to <= 0.0: 0.0 -- alpha : 0.06522924308576186 -- lamda : 0.01
t:12576 -- u starts to <= 0.0: 0.0 -- alpha : 0.0606399822493871 -- lamda : 18.0
t:13009 -- u starts to <= 0.0: 0.0 -- alpha : 0.055566564227172335 -- lamda : 18.0
t:12220 -- u starts to <= 0.0: 0.0 -- alpha : 0.0655543994544671 -- lamda : 7.278418431870787
t:12740 -- u starts to <= 0.0: 0.0 -- alpha : 0.05861160904723629 -- lamda : 15.55093432914834
t:12237 -- u starts to <= 0.0: 0.0 -- alpha : 0.06529957906717802 -- lamda : 18.0


100%|██████████| 1400/1400 [57:13<00:00,  2.45s/it]

t:11241 -- u starts to <= 0.0: 0.0 -- alpha : 0.08435200662935487 -- lamda : 8.009012045927394
STOP CODE:  Stay unchanged in the last 6 iterations
> SA ENDS....... 



In [26]:
params_opitim.head()

,key_idx,alpha,delta,labda,initial_loss,final_loss,avg_loss
0,0,-0.007870,1,18.00000,99.047947,64.524913,6.452491
1,1,-0.041023,1,0.35904,754.827049,198.546584,9.927329
2,2,-0.008125,1,18.00000,343.478827,219.344713,6.646809
3,3,-0.009420,1,18.00000,158.898107,98.758510,6.583901
4,4,-0.036422,1,0.01000,5803.451641,643.136072,13.125226


In [27]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_delta_path, header=True, encoding="utf-8",index=False)

# 4. append with data key
1. 根据需要更改一下，总之只执行1次
2. 添加data key以及duration等信息

In [7]:
params_opitim = pd.read_csv(params_opitim_delta_path, encoding="utf-8")
params_opitim_withsetting = pd.concat([data_key,params_opitim],axis = 1)

In [11]:
max_T_tmp = []
for i in range(0,N_uniq_auction):

    # get optimized params
    alpha, labda, delta = params_opitim.iloc[i][0],params_opitim.iloc[i][1],params_opitim.iloc[i][2]

    # get i_th data_key
    key_i = get_key_from_index(i)
    # extract data with same `key_i` into a table
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value
    max_T_tmp.append(max_T)

params_opitim_withsetting['max_T'] = np.array(max_T_tmp)
params_opitim_withsetting['T'] = np.array((params_opitim_withsetting.retail - params_opitim_withsetting.bidfee) / params_opitim_withsetting.bidincrement,dtype=int)

params_opitim_withsetting.to_csv(params_opitim_delta_wset_path, header=True, encoding="utf-8",index=False)
params_opitim_withsetting.head(5)

,product_id,bidincrement,bidfee,retail,key_idx,alpha,delta,labda,initial_loss,final_loss,avg_loss,max_T,T
0,10009881,0.15,0.75,169.99,0,-0.050708,1,0.368038,766.411406,202.448793,9.640419,831,1128.266667
1,10006115,0.15,0.75,499.99,1,-0.008440,1,18.000000,340.195808,218.717313,6.627797,930,3328.266667
2,10007148,0.15,0.75,299.99,2,-0.036130,1,0.300196,5803.451641,616.843421,12.588641,1531,1994.933333
3,10007263,0.15,0.75,89.99,3,-0.046248,1,2.811367,4181.687406,2734.820053,6.670293,523,594.933333
4,10010575,0.15,0.75,59.99,4,-0.025051,1,3.781233,266.034143,238.850389,4.593277,150,394.933333


# 5. TEST
1. 对之前infer不出来的一些setting拿出来，看看在一组固定的参数下，u的变化
    - 其实当max_T过大的时候，u会变成负数

In [41]:
viz = Visdom(env='001')
# 改这个pls
i=9
# 参数固定下
alpha = 0.002
labda = 0.38

key_i = get_key_from_index(i)
data_i = select_data_fromkey(key_i)
data_i.reset_index(drop=True,inplace=True)

T_i = data_i['N'].astype(int)
v = float(data_i['retail'].unique())
d = float(data_i['bidincrement'].unique())
b = float(data_i['bidfee'].unique())
max_T = int(max(T_i))                  # max duration value
U_i = [0] * (max_T + 1)
U_i[0] = 1

for t in range(1,max_T+1):

    U_i[t] = f_Equi(t, v, d, b, alpha, labda)

viz.line([0.0]*(max_T+1),[0]*(max_T+1),win = 'Loss of '+str(i),env='001',opts= dict(title='Loss of '+str(i)))
viz.line(np.array(U_i),np.arange(0,max_T+1),win = 'Loss of '+str(i), env='001',update='append')

Setting up a new session...


tmp starts to < 0 2498
tmp starts to < 0 2499
tmp starts to < 0 2500
tmp starts to < 0 2501
tmp starts to < 0 2502
tmp starts to < 0 2503
tmp starts to < 0 2504
tmp starts to < 0 2505
tmp starts to < 0 2506
tmp starts to < 0 2507
tmp starts to < 0 2508
tmp starts to < 0 2509
tmp starts to < 0 2510
tmp starts to < 0 2511
tmp starts to < 0 2512
tmp starts to < 0 2513
tmp starts to < 0 2514
tmp starts to < 0 2515
tmp starts to < 0 2516
tmp starts to < 0 2517
tmp starts to < 0 2518
tmp starts to < 0 2519
tmp starts to < 0 2520
tmp starts to < 0 2521
tmp starts to < 0 2522
tmp starts to < 0 2523
tmp starts to < 0 2524
tmp starts to < 0 2525
tmp starts to < 0 2526
tmp starts to < 0 2527
tmp starts to < 0 2528
tmp starts to < 0 2529
tmp starts to < 0 2530
tmp starts to < 0 2531
tmp starts to < 0 2532
tmp starts to < 0 2533
tmp starts to < 0 2534
tmp starts to < 0 2535
tmp starts to < 0 2536
tmp starts to < 0 2537
tmp starts to < 0 2538
tmp starts to < 0 2539
tmp starts to < 0 2540
tmp starts 

'Loss of 9'

In [31]:
#print(viz.get_env_list())

19.8

In [21]:

# 保存整个环境
viz.save(envs=['Loss of PT'])

'["Loss of PT"]'